In [1]:
import tensorflow as tf
import wandb
from wandb.integration.keras import WandbMetricsLogger
from tensorflow import keras
import numpy as np
import math

In [2]:
sweep_config = {
    'method' : 'grid',
    'metric' : {
        'name' : 'val_accuracy',
        'goal' : 'maximize'
    },
    'parameters' : {
        'learning_rate' : {'values': [0.001, 0.0001, 0.00001]},
        'batch_size' : {'values': [32, 64, 128]},
        'img_size' : {'values': [64, 128, 224]},
        'epochs' : {'values': [5, 10]},
        'hidden_nodes' : {'values': [64, 128]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="5-flowers-project")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: iiskipi1
Sweep URL: https://wandb.ai/hussaintoori/5-flowers-project/sweeps/iiskipi1


In [3]:
def train():
    wandb.init()
    config = wandb.config

    IMG_HEIGHT = config.img_size
    IMG_WIDTH = config.img_size
    IMG_CHANNELS = 3

    CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]


    def read_and_decode(filename, resize_dims):
      # 1. Read the raw file
      img_bytes = tf.io.read_file(filename)
      # 2. Decode image data
      img = tf.image.decode_jpeg(img_bytes, channels = IMG_CHANNELS)
      # 3. Convert pixel values to floats in [0, 1]
      img = tf.image.convert_image_dtype(img, tf.float32)
      # 4. Resize the image to match desired dimensions
      img = tf.image.resize(img, resize_dims)
      return img

    def parse_csvline(csv_line):
      # record_defaults specify the data types for each column
      record_default = ["", ""]
      filename, label_string = tf.io.decode_csv(csv_line, record_default)

      # Load the image
      img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])

      # Convert label string to integer based on the CLASS_NAMES index
      label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
      return img, label

    # Define datasets
    train_dataset = (
      tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
      .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
      .batch(config.batch_size)
      .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
      tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
      .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
      .batch(config.batch_size)
      .prefetch(tf.data.AUTOTUNE)
    )

    model = keras.Sequential([
      keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
      keras.layers.Dense(config.hidden_nodes, activation = "relu"), #additional layer of neural network.
      keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
    ])

    model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=config.learning_rate), # learning rate could be set here. by default it's 0.001 here
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"]
    )

    model.fit(
      train_dataset,
      validation_data = eval_dataset,
      epochs = config.epochs,
      callbacks = WandbMetricsLogger(log_freq=5)
    )


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: q3scz0gk with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_nodes: 64
wandb: 	img_size: 64
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: hussainturi (hussaintoori) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
    104/Unknown 184s 2s/step - accuracy: 0.3149 - loss: 2.1008

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


104/104 ━━━━━━━━━━━━━━━━━━━━ 206s 2s/step - accuracy: 0.3152 - loss: 2.0971 - val_accuracy: 0.3811 - val_loss: 1.4800
Epoch 2/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.3966 - loss: 1.4255 - val_accuracy: 0.4270 - val_loss: 1.3013
Epoch 3/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.4334 - loss: 1.3389 - val_accuracy: 0.4568 - val_loss: 1.2415
Epoch 4/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.4722 - loss: 1.2518 - val_accuracy: 0.4838 - val_loss: 1.2148
Epoch 5/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 96s 919ms/step - accuracy: 0.5012 - loss: 1.1827 - val_accuracy: 0.4973 - val_loss: 1.2063


...

wandb: Agent Starting Run: 5a7uiv6i with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_nodes: 64
wandb: 	img_size: 128
wandb: 	learning_rate: 0.001


Epoch 1/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 88s 829ms/step - accuracy: 0.2659 - loss: 7.1184 - val_accuracy: 0.3270 - val_loss: 1.5198
Epoch 2/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 141s 824ms/step - accuracy: 0.2496 - loss: 1.5966 - val_accuracy: 0.3162 - val_loss: 1.6007
Epoch 3/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 89s 856ms/step - accuracy: 0.3164 - loss: 1.5589 - val_accuracy: 0.2595 - val_loss: 1.6265
Epoch 4/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.2623 - loss: 1.5915 - val_accuracy: 0.2405 - val_loss: 1.5986
Epoch 5/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 85s 808ms/step - accuracy: 0.2565 - loss: 1.5834 - val_accuracy: 0.2243 - val_loss: 1.6028


batch/accuracy,▁▂▃▄▅▆▆▆▆█▃▃▃▄▄▄▄▄▆▇▇▇▆▄▃▄▃▄▄▄▅▅▅▄▄▅▅▅▅▄
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,▁█▇▆▅▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▆▁█▄▁
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▁▁▁▁
epoch/val_accuracy,█▇▃▂▁
epoch/val_loss,▁▆█▆▆
batch/accuracy,0.26269


wandb: Agent Starting Run: u0qqvum1 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_nodes: 64
wandb: 	img_size: 128
wandb: 	learning_rate: 0.0001


Epoch 1/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 86s 807ms/step - accuracy: 0.3334 - loss: 1.5974 - val_accuracy: 0.4514 - val_loss: 1.3177
Epoch 2/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 142s 821ms/step - accuracy: 0.4347 - loss: 1.3349 - val_accuracy: 0.4541 - val_loss: 1.2507
Epoch 3/5
 17/104 ━━━━━━━━━━━━━━━━━━━━ 57s 665ms/step - accuracy: 0.4436 - loss: 1.2830